In [5]:
import pandas as pd

# If you need specific options, you can add parameters like:
df = pd.read_csv('src/data/da_constraints_20250101.csv',
                 encoding='utf-8',  # specify encoding if needed
                 sep=',',           # specify delimiter
                 header=0) 

In [6]:
# Get unique counts for Monitored Facility
monitored_counts = df['Monitored Facility'].value_counts()
print("\nMonitored Facility Counts:")
print(monitored_counts)

# Get unique counts for Contingent Facility
contingent_counts = df['Contingent Facility'].value_counts()
print("\nContingent Facility Counts:")
print(contingent_counts)

# Optional: If you want to see total number of unique values
print(f"\nTotal unique Monitored Facilities: {df['Monitored Facility'].nunique()}")
print(f"Total unique Contingent Facilities: {df['Contingent Facility'].nunique()}")


Monitored Facility Counts:
Monitored Facility
LN TEXAS_CO -               7377
LN GRACMONT - ANADARKO      7022
Multi-Element Constraint    6146
LN WR_SMKHL - SUMM          3699
XFMR POTTER_S - POTTER_S    3240
                            ... 
LN REDHILLS - ELK-CITY         1
LN CUSTER - REDHILLS           1
LN SUB_M - SHRNKRD             1
LN SMITH-1 - PHLBUR1           1
LN 59TH - ELPA                 1
Name: count, Length: 978, dtype: int64

Contingent Facility Counts:
Contingent Facility
BASE                                  33087
WFEC CSWS:WASHIT1 SW_STA:138:1:6       3737
OKGE:BROWN2 BODLE CANEY1:138:7:34      3259
SPS:RDRUNNER XF TRT:345 115:3:17       2720
GRDA OKGE:CLEVLND7 SONR1:345:1:10      2643
                                      ...  
WAUE:POPLARWP WOLFPT:115:2:6              1
KACY KCPL:BARBER TERRACE:161:1:12         1
WAUE:ANTELOP LELANDO:345:1:11             1
KCPL:LEVEE NRTHEAST:161:1:11              1
WAUE:LELANDO2 LELANDO:345 230:4:14        1
Name: count, Lengt

In [13]:
import pandas as pd
import networkx as nx

def parse_facility(facility_str: str):
    """
    Parse facility string to extract nodes, voltages, and company.
    
    Example inputs:
    - "LN FLOURNOY - WOOLWORT" -> (["FLOURNOY", "WOOLWORT"], [], None)
    - "CSWS:LEASIDE SWSHREV:138:1:9" -> (["LEASIDE", "SWSHREV"], [138.0], "CSWS")
    - "OKGE:WDWRD1 XF A:138 69:1:5" -> (["WDWRD1", "XF A"], [138.0, 69.0], "OKGE")
    - "WAUE:BELFELD XF KU1A3:345 230 13.8" -> (["BELFELD", "XF KU1A3"], [345.0, 230.0, 13.8], "WAUE")
    - "SPS:HOBBS:COMBINEDCYCLE" -> (["HOBBS"], [], "SPS")
    """
    # Parse detailed format (company:node1 node2:voltage:...)
    if ':' in facility_str:
        parts = facility_str.split(':')
        company = parts[0]
        nodes = parts[1].split()
        
        # Handle multiple voltage levels, including decimals
        voltages = []
        if len(parts) > 2:
            try:
                voltages = [float(v) for v in parts[2].split()]
            except ValueError:
                # If voltage parsing fails, just continue with empty voltages
                pass
            
        return nodes, voltages, company
        
    # Parse simple format (LN/XFMR NODE1 - NODE2)
    else:
        nodes = [n.strip() for n in facility_str.split('-')]
        if nodes[0].startswith(('LN ', 'XFMR ')):
            nodes[0] = nodes[0].split(' ', 1)[1].strip()
        return nodes, [], None

def add_constraint_to_network(row: pd.Series):
    """Add a single constraint's information to the network."""
    # Parse monitored facility
    mon_nodes, mon_voltages, mon_company = self.parse_facility(row['Monitored Facility'])
    
    # Parse contingent facility if not BASE
    if row['Contingent Facility'] != 'BASE':
        cont_nodes, cont_voltages, cont_company = self.parse_facility(row['Contingent Facility'])
    else:
        cont_nodes, cont_voltages, cont_company = [], [], None
    
    # Add nodes and edges to network
    for nodes, voltages, company in [(mon_nodes, mon_voltages, mon_company)]:
        #    (cont_nodes, cont_voltages, cont_company)]:
        if not nodes:
            continue
            
        # Add nodes with attributes
        for node in nodes:
            if node not in self.G:
                self.G.add_node(node, voltages=set(), companies=set())
            
            if voltages:
                self.G.nodes[node]['voltages'].update(voltages)
            if company:
                self.G.nodes[node]['companies'].add(company)
        
        # Add edge between nodes if it's a pair
        if len(nodes) == 2:
            self.G.add_edge(nodes[0], nodes[1], 
                          voltages=set(voltages),
                          company=company)

def build_network(constraints_df: pd.DataFrame):
    """Build network from constraints dataframe."""
    for _, row in constraints_df.iterrows():
        self.add_constraint_to_network(row)

In [18]:
unique_edges = df['Contingent Facility'].unique()

# Create empty lists to store data
sources = []
sinks = []
voltages = []
companies = []

# Parse each unique facility
for edge in unique_edges:
    source_sink, voltage, company = parse_facility(edge)
    print(source_sink)
    if len(source_sink) == 2:
        sources.append(node[0])
        sinks.append(node[1])
        voltages.append(voltage if voltage else None)
        companies.append(company if company else None)

# Create DataFrame from the lists
facility_df = pd.DataFrame({
    'Source': sources,
    'Sink': sinks,
    'Voltage': voltages,
    'Company': companies
})

facility_df

['BASE']
['BLACKBRY', 'JASPER7']
['WAHPETN', 'HANKSON']
['SEMINOLE']
['NICHSUB', 'GRAPVINE']
['MCC2', 'MTGY']
['SHAM', 'KIRBY_SU']
['SETAB1', 'MINGO1']
['CHUB_LK', 'HELENAMN']
['RAUN', 'FT_CAL']
['SIBLEY', 'OVER']
['STL4391', 'JOP3891']
['CHAR_CK', 'PATENT_G']
['HOYT', 'STRA']
['BORDER', 'TUCO']
['MCCLAI', 'PLVAL']
['MACON345', 'AXTELL1']
['GRACMONT', 'ANADARKO']
['JUD-LR1', 'CRKCREEK']
['VALLIANT', 'PITTSB9']
['CLEVLND7', 'SONR1']
['WASHIT1', 'SW_STA']
['SEMINOLE', 'PITTSB9']
['MINCO', 'CIMARRON']
['GENTRY', 'NODAWAY', 'FAIRPORT']
['GAVINS', 'SPIRITM']
['MULGRE2', 'CIRC']
['BORDER', 'TUCO']
['N345', 'NEORDG']
['FORMNWA', 'ELLIOTTW']
['SPIRITM', 'MANNING2']
['COLBY3', 'MINGO1']
['HOLWD', 'PLVAL']
['SEMINOLE', 'XF', '1_3']
['WAYSID_W', 'STEGALL']
['SUB3740', 'SUB3455']
['SCOTSBLF', 'VICTRY_H']
['SUB1280', 'SUB1263']
['SHELDON', 'FOLSOM']
['CEDAR_CR', 'SHWNMSN5']
['JERICHO_', 'KIRBY_SU']
['WAGENER', 'SUB3454']
['BROWN2', 'BODLE', 'CANEY1']
['HUGOPP4', 'ATOKA2']
['W_GARDNR', 'LACYGNE']
['

,Source,Sink,Voltage,Company
0,M,I,[345.0],AECI
1,M,I,"[230.0, 115.0, 41.6]",OTP
2,M,I,[230.0],SPS
3,M,I,[345.0],AECI AMRN
4,M,I,[115.0],CSWS SPS
...,...,...,...,...
638,M,I,[161.0],OKGE
639,M,I,[138.0],OKGE
640,M,I,[115.0],WR
641,M,I,[115.0],WAUE
